In [1]:
# Parameters
cell_type = "UTC"


In [2]:
import sys
import os

import anndata as ad

import pandas as pd
import numpy as np

from pyprojroot import here

import re

from tqdm import tqdm

In [3]:
def load_sorted_shap_values_fnames(
    cell_type: str = '',
    run_name: str = 'run1'):

    dirpath = here(f"inflammabucket_bkp/03_downstream_analysis/05_SHAP/results/04_shap/shap_vals")
    fname_regex = f'^{run_name}_{cell_type}_shap_values_' + r'(\d+)'
    results_batches = [
        (fname, int(re.search(fname_regex, fname).group(1)))
        for fname in os.listdir(dirpath)
        if re.search(fname_regex, fname)]

    sorted_files = sorted(results_batches, key=lambda x: x[1])

    sorted_filenames = [os.path.join(dirpath, filename) for filename, _ in sorted_files]

    return sorted_filenames

In [4]:
shap_val_list = [np.load(fname)['shap_values'] for fname in tqdm(load_sorted_shap_values_fnames(cell_type))] 


  0%|          | 0/12 [00:00<?, ?it/s]

  8%|▊         | 1/12 [00:03<00:36,  3.35s/it]

 17%|█▋        | 2/12 [00:07<00:35,  3.54s/it]

 25%|██▌       | 3/12 [00:10<00:31,  3.50s/it]

 33%|███▎      | 4/12 [00:13<00:26,  3.27s/it]

...

 75%|███████▌  | 9/12 [00:29<00:09,  3.31s/it]

 83%|████████▎ | 10/12 [00:32<00:06,  3.32s/it]

 92%|█████████▏| 11/12 [00:35<00:03,  3.28s/it]

100%|██████████| 12/12 [00:37<00:00,  2.81s/it]

100%|██████████| 12/12 [00:37<00:00,  3.11s/it]

In [5]:
adata = ad.read_h5ad(here(f'inflammabucket_bkp/03_downstream_analysis/05_SHAP/data/{cell_type}_adataMerged_SPECTRAgenes.log1p.h5ad'), backed='r')

In [6]:
## Gene symbols and Disease labels
symbols_df = pd.read_pickle(here('inflammabucket_bkp/03_downstream_analysis/04_selected_gene_list.pkl'))
symbols_sorted = symbols_df.loc[adata.var_names].symbol.values

DISEASES = ['BRCA', 'CD', 'COPD', 'COVID', 'CRC', 'HBV', 'HIV', 'HNSCC', 'MS', 'NPC', 'PS', 'PSA', 'RA', 'SLE', 'UC', 'asthma', 'cirrhosis', 'flu', 'healthy', 'sepsis']

diseaseDict = dict()
for d in DISEASES:
    diseaseDict[d] = []

In [7]:
shape_values_matrix = np.concatenate(shap_val_list)

In [8]:
for idx, values in tqdm(adata.obs.groupby('sampleID', observed=True).indices.items()):
    geneXdisease_sample_i = pd.DataFrame(shape_values_matrix[values].mean(0))
    geneXdisease_sample_i.columns = DISEASES
    geneXdisease_sample_i.index = symbols_sorted
    for d in geneXdisease_sample_i.columns:
        diseaseDict[d].append(pd.DataFrame.from_dict({idx:geneXdisease_sample_i[d]}))

  0%|          | 0/813 [00:00<?, ?it/s]

  3%|▎         | 22/813 [00:00<00:03, 217.72it/s]

  7%|▋         | 60/813 [00:00<00:02, 312.30it/s]

 12%|█▏        | 100/813 [00:00<00:02, 349.53it/s]

 17%|█▋        | 135/813 [00:00<00:02, 319.23it/s]

...

 91%|█████████ | 738/813 [00:02<00:00, 180.48it/s]

 94%|█████████▎| 761/813 [00:02<00:00, 188.97it/s]

 96%|█████████▌| 782/813 [00:02<00:00, 191.38it/s]

 99%|█████████▉| 803/813 [00:03<00:00, 188.34it/s]

100%|██████████| 813/813 [00:03<00:00, 256.03it/s]

In [9]:
for d in tqdm(DISEASES):
    pd.concat(diseaseDict[d], axis=1).to_csv(here(f'03_downstream_analysis/08_gene_importance/new_shap_plots/results/SHAP_AVGsamples/SHAP_AVGsample_{cell_type}_{d}.csv'))

  0%|          | 0/20 [00:00<?, ?it/s]

  5%|▌         | 1/20 [00:01<00:20,  1.09s/it]

 10%|█         | 2/20 [00:01<00:15,  1.15it/s]

 15%|█▌        | 3/20 [00:02<00:14,  1.15it/s]

 20%|██        | 4/20 [00:03<00:15,  1.04it/s]

...

 85%|████████▌ | 17/20 [00:15<00:02,  1.12it/s]

 90%|█████████ | 18/20 [00:15<00:01,  1.14it/s]

 95%|█████████▌| 19/20 [00:17<00:01,  1.03s/it]

100%|██████████| 20/20 [00:18<00:00,  1.07s/it]

100%|██████████| 20/20 [00:18<00:00,  1.08it/s]